<a href="https://colab.research.google.com/github/Oluchi-Otuadinma/QuickCode/blob/main/PDF_Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Python Script to Extract PDF Content and Save to Excel

In [ ]:
pip install pymupdf camelot-py[cv] pandas openpyxl

In [ ]:
import fitz  # PyMuPDF
import camelot
import pandas as pd
import re

# Set PDF Path
pdf_path = "your_file.pdf"  # Replace with your PDF path, make sure you keep in the ""

# Extract Text and Numbers
doc = fitz.open(pdf_path)
text_blocks = []
number_blocks = []

for i, page in enumerate(doc):
    text = page.get_text()
    text_blocks.append({'type': 'text', 'page': i+1, 'content': text})

    # Extract numbers using regex (integers or decimals)
    numbers = re.findall(r'\b\d+(?:\.\d+)?\b', text)
    number_blocks.append({'type': 'number', 'page': i+1, 'content': ', '.join(numbers)})

# Convert to DataFrames
text_df = pd.DataFrame(text_blocks)
number_df = pd.DataFrame(number_blocks)

# Extract Tables Using Camelot
tables = camelot.read_pdf(pdf_path, pages="all", flavor='stream')  # or use flavor='lattice' depending on your border style
table_frames = []

for i, table in enumerate(tables):
    df = table.df
    df.insert(0, 'table_id', i+1)
    df.insert(0, 'type', 'table')
    table_frames.append(df)

if table_frames:
    tables_df = pd.concat(table_frames, ignore_index=True)
else:
    tables_df = pd.DataFrame(columns=['type', 'table_id'])

# Combine All Extracted Data
combined_df = pd.concat([text_df, number_df, tables_df], ignore_index=True, sort=False)

# Step 5: Save to Excel (single sheet)
combined_df.to_excel("extracted_data.xlsx", index=False)


#Output will be "extracted_data.xlsx"

### It contains:

*   Text blocks with page numbers
*   Lists of numeric values from each page
*   Structured tables (each row labeled as type "table")


In [ ]:
#---Example PDF input---

Monthly Report - January

Summary:
Revenue increased by 12.5% this month.
Total Customers: 1,024
New Signups: 325

Table 1 - Product Sales:
| Product | Units Sold | Revenue |
|---------|------------|---------|
| A       | 120        | 2,400   |
| B       | 85         | 1,700   |
| C       | 200        | 4,000   |


Customer Feedback (Page 2):

- “Great service.” — John
- “Prices are too high.” — AliceT

Table 2 - Complaint Categories:
| Category     | Count |
|--------------|-------|
| Pricing      | 45    |
| Shipping     | 23    |
| Product Defect | 10  |

In [ ]:
#---Expected Output Excel (extracted_data.xlsx)---

| type   | page | content                                                                                    | table\_id | Product | Units Sold | Revenue | Category       | Count |
| ------ | ---- | ------------------------------------------------------------------------------------------ | --------- | ------- | ---------- | ------- | -------------- | ----- |
| text   | 1    | Monthly Report - January\n\nSummary:\nRevenue increased by 12.5% this month.\n...          |           |         |            |         |                |       |
| number | 1    | 12.5, 1024, 325, 120, 2400, 85, 1700, 200, 4000                                            |           |         |            |         |                |       |
| table  |      |                                                                                            | 1         | A       | 120        | 2400    |                |       |
| table  |      |                                                                                            | 1         | B       | 85         | 1700    |                |       |
| table  |      |                                                                                            | 1         | C       | 200        | 4000    |                |       |
| text   | 2    | Customer Feedback (Page 2):\n\n- “Great service.” — John\n- “Prices are too high.” — Alice |           |         |            |         |                |       |
| number | 2    | 45, 23, 10                                                                                 |           |         |            |         |                |       |
| table  |      |                                                                                            | 2         |         |            |         | Pricing        | 45    |
| table  |      |                                                                                            | 2         |         |            |         | Shipping       | 23    |
| table  |      |                                                                                            | 2         |         |            |         | Product Defect | 10    |
